# Dataload

The following notebook walks through how to load prerequisite data into ESCALATE. This presumes all database tables are completely empty to start.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import escalateclient
import importlib
import pandas as pd

In [ ]:
importlib.reload(escalateclient)
server_url = 'http://localhost:8000'
# server_url = 'https://escalate.cs.haverford.edu/test_server'
username = 'vshekar'
password = 'copperhead123'
client = escalateclient.ESCALATEClient(server_url, username, password)

# Users and Organizations

In [ ]:
#List of statuses to create
statuses = ["inactive", "active", "test", "do_not_use", "prototype"]    

#Create statuses 
for s in statuses:
    status_response = client.get_or_create(endpoint='status', data={'description': s})

#Get active status 
active_status = client.get_or_create(endpoint='status', data={'description': 'active'})

# Materials and Inventory

In [ ]:
#List of material identifier definitions to create
material_identifier_defs =  ["SMIRKS", "Chemical_Name", "InChI", "SMILES", "Molecular_Formula", "SMARTS", "RInChI", "InChIKey", "Abbreviation"]

#Create material identifier definitions 
for mid in material_identifier_defs:
    mid_response = client.get_or_create(endpoint='material-identifier-def', data={'description': mid})

#List of material types to create
material_types =  ["solvent","antisolvent", "acid", "organic", "inorganic", "polymer"]

#Create material types 
for mt in material_types:
    mt_response = client.get_or_create(endpoint='material-type', data={'description': mt})

In [ ]:
chem_list = pd.read_csv('Chemicals List.csv')
chem_list = chem_list.fillna('')
chem_list['Material type'] = chem_list['Material type'].str.replace('Gas', 'flux')

for i, row in chem_list.iterrows():
    chemical_name = row['Chemical Name']
    material_types = row['Material type'].lower().split(',')
    mt_responses = []
    for mt in material_types:
        mt_responses.append(client.get_or_create('material-type', data={'description': mt})[0]) # Get material type from a separate database table
    # Add material to db
    material_data = {'description': chemical_name, 'material_type': [mtr['url'] for mtr in mt_responses], 'material_class':'model'}
    material_response = client.get_or_create('material', data=material_data)
    # Add material to inventory
    if row['Inventory Name']:
        description = f"{chemical_name} {row['Inventory Name']}"
    else:
        description = chemical_name
    im_data = {
        "description": description,
        "part_no": f"{row['CAS Num']}",
        "phase": row['Phase'].lower(),
        "inventory": nl_inventory_response[0]['url'], #Associates inventory material with Neilson lab inventory
        "material": material_response[0]['url'] #Associates inventory material with material
    }
    im_response = client.get_or_create('inventory-material', data=im_data)

# Vessels

In [ ]:
vessels_list = pd.read_csv('load_opentrons_vessels.csv')
vessels_list = vessels_list.fillna('')

for i, row in vessels_list.iterrows():
    vessel_name = row['description']
    vol = row["total_volume"]
    #Enter volume as a value field
    if len(vol.split()) > 1:
        val = vol.split()[0]
        unit = vol.split()[1]
        total_volume = {"value": val, "unit": unit, "type": "num"}
    else:
        total_volume = None
    
    well_number = row["well_number"]
    column_order = row["column_order"]

    vessel_data = {'description': vessel_name,
                    'total_volume': total_volume,
                    'well_number': well_number,
                    'column_order': column_order,
                    'status': active_status[0]['url'],
                    }
    # Add vessel data to db
    vessel_response = client.get_or_create('vessel', data=vessel_data)

# Experiment-related Definitions (Actions, Parameters, Properties)

### Properties

In [ ]:
# Create default values
volume_value = {"value": 0, "unit": "ml", "type": "num"}
zero_ml_data = {"description": "Zero ml", "nominal_value": volume_value, "actual_value": volume_value,}
zero_ml_response = client.get_or_create(endpoint='default-values', data=zero_ml_data)

mass_value = {"value": 0, "unit": "g", "type": "num"}
zero_g_data = {"description": "Zero g", "nominal_value": mass_value, "actual_value": mass_value,}
zero_g_response = client.get_or_create(endpoint='default-values', data=zero_g_data)

concentration_value = {"value": 0, "unit": "M", "type": "num"}
zero_M_data = {"description": "Zero M", "nominal_value": concentration_value, "actual_value": concentration_value,}
zero_M_response = client.get_or_create(endpoint='default-values', data=zero_M_data)

# List of properties to create
property_data={"total volume": zero_ml_response,
                "dead volume": zero_ml_response,
                "concentration": zero_M_response,
                "amount": zero_g_response}

# Create properties
for key, val in property_data:
    prop_data = {
                "description": key,
                "property_def_class": "extrinsic",
                "short_description": key,
                "default_value": val[0]['url'], #Associates property with default value
            }
    prop_response = client.get_or_create(endpoint='property-template', data=prop_data)

### Parameters

In [ ]:
params_list = pd.read_csv('parameter_def.csv', sep="\t")
params_list = params_list.fillna('')


for i, row in params_list.iterrows():
    description = row["description"]
    type_ = row["type"]
    value_from_csv = str(row["value"])
    unit = row["unit"]
    required = row["required"]
    unit_type = row["unit_type"]
    params_data = {
        "description": description,
        "default_val": {'value': float(value_from_csv), 'unit': unit, 'type': type_},
        "unit_type": unit_type,
        "required": required,
        "status": active_status[0]['url'],
    }
    params_response = client.get_or_create('parameter-def', data=params_data)

### Actions